In [1]:
!pip install -q bitsandbytes accelerate peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
import pandas as pd
import random
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

2025-11-22 08:36:19.084347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763800579.296064      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763800579.349470      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
def clean_answer(a):
    if pd.isna(a) or a == "":
        return ""
    a = str(a).strip()
    # 移除首尾标点（保留内部连字符、空格）
    a = re.sub(r"^[^\w\s\-]+|[^\w\s\-]+$", "", a)
    return a.strip()

# 加载原始数据
build_data = pd.read_csv("/kaggle/input/train-data/build_data.csv", delimiter="\t", quoting=3)

# 清洗
build_data = build_data.dropna(subset=["question_en", "answer_en"])
build_data["answer_clean"] = build_data["answer_en"].apply(clean_answer)
build_data = build_data[build_data["answer_clean"] != ""]
build_data = build_data.reset_index(drop=True)

questions = build_data["question_en"].tolist()
answers = build_data["answer_clean"].tolist()
all_answers = answers.copy()

# 构造样本
train_samples = []
random.seed(42)

for q, correct_a in zip(questions, answers):
    # 正样本
    train_samples.append({
        "input_text": f'The answer to "{q}" is: {correct_a}.',
        "output": "yes"
    })
    
    # 负样本：随机选一个不同的答案
    wrong_candidates = [a for a in all_answers if a != correct_a]
    if wrong_candidates:
        wrong_a = random.choice(wrong_candidates)
        train_samples.append({
            "input_text": f'The answer to "{q}" is: {wrong_a}.',
            "output": "no"
        })

# 打乱并划分
random.shuffle(train_samples)
df_full = pd.DataFrame(train_samples)
train_df, val_df = train_test_split(df_full, test_size=0.1, random_state=42)


In [4]:
model_id = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# 4-bit 量化配置
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)
model = prepare_model_for_kbit_training(model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
# LoRA 配置
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 33,030,144 || all params: 4,055,498,240 || trainable%: 0.8145


In [6]:
EOS_TOKEN = tokenizer.eos_token
# === 数据格式化函数 ===
def formatting_prompts_func(examples):
    inputs = examples["input_text"]
    outputs = examples["output"]
    texts = [inp + " " + out + EOS_TOKEN for inp, out in zip(inputs, outputs)]
    return {"text": texts}

train_dataset = Dataset.from_pandas(train_df).map(formatting_prompts_func, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/3551 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

In [7]:
os.makedirs('/kaggle/working/checkpoint', exist_ok=True)
training_args = TrainingArguments(
    output_dir="/kaggle/working/checkpoint",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=True,
    bf16=False,
    seed=3407,
    report_to="none",
)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


Adding EOS to train dataset:   0%|          | 0/3551 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3551 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3551 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/395 [00:00<?, ? examples/s]

In [8]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,0.537200,0.519503,0.539622,84631.000000,0.833015
2,0.476400,0.495380,0.505211,169262.000000,0.842986
3,0.450300,0.487332,0.478788,253893.000000,0.843836


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1332, training_loss=0.5317609259137163, metrics={'train_runtime': 1259.4945, 'train_samples_per_second': 8.458, 'train_steps_per_second': 1.058, 'total_flos': 7029663136711680.0, 'train_loss': 0.5317609259137163, 'epoch': 3.0})

In [9]:
model.save_pretrained("/kaggle/working/final_lora")
tokenizer.save_pretrained("/kaggle/working/final_lora")

('/kaggle/working/final_lora/tokenizer_config.json',
 '/kaggle/working/final_lora/special_tokens_map.json',
 '/kaggle/working/final_lora/chat_template.jinja',
 '/kaggle/working/final_lora/vocab.json',
 '/kaggle/working/final_lora/merges.txt',
 '/kaggle/working/final_lora/added_tokens.json',
 '/kaggle/working/final_lora/tokenizer.json')

In [10]:
test_df = pd.read_csv("/kaggle/input/train-data/ex_questions1.csv", delimiter="\t", quoting=3)
test_prompts = test_df["input_text"].astype(str).tolist()
test_ids = test_df["id"].tolist()
test_options = test_df["option_letter"].tolist()
model.eval()
model.half()

predictions_binary = []

for prompt in tqdm(test_prompts, desc="Inference"):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding=False,
        add_special_tokens=True,
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,      # 只需生成 "yes" 或 "no"
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        )
    
    generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    pred_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip().lower()
    
    # 判断输出
    if pred_text.startswith("yes"):
        pred = 1
    elif pred_text.startswith("no"):
        pred = 0
    else:
        # 无法识别时默认为 0（保守）
        pred = 0
    
    predictions_binary.append(pred)

Inference: 100%|██████████| 592/592 [05:46<00:00,  1.71it/s]


In [11]:
# === 保存结果 ===
result_df = pd.DataFrame({
    "id": test_ids,
    "answer_en": predictions_binary,
    "option": test_options
})

result_df.to_csv("/kaggle/working/mcq_instruction_tuning.csv", index=False, sep='\t')

In [12]:
temp_df = pd.DataFrame({
    "id": test_ids,
    "pred": predictions_binary,
    "option": test_options
})

correct_options = temp_df[temp_df["pred"] == 1]

all_ids = pd.DataFrame({"id": sorted(temp_df["id"].unique())})

def get_answer(qid):
    matches = correct_options[correct_options["id"] == qid]
    if not matches.empty:
        return matches.iloc[0]["option"]
    else:
        return " "

all_ids["answer_en"] = all_ids["id"].apply(get_answer)

def extract_number(id_str):
    try:
        return int(id_str.split('_')[-1])
    except:
        return float('inf')

all_ids['sort_key'] = all_ids['id'].apply(extract_number)
final_result = all_ids.sort_values('sort_key').drop(columns=['sort_key']).reset_index(drop=True)

# === 保存 ===
final_result.to_csv("/kaggle/working/mcq_instruction_tuning.csv", index=False, sep='\t')